In [1]:
# workaround for incredibly stupid vscode on snap bug
from os import environ
krb5ccname = environ["KRB5CCNAME"]
if "hostfs" in krb5ccname:
    krb5ccname = krb5ccname.replace("/var/lib/snapd/hostfs", "")
    environ["KRB5CCNAME"] = krb5ccname

In [2]:
import ROOT
from analysis_framework import Dataset
from ObjectSelectionHelper import ObjectSelectionHelper, make_lvec_M, make_lvec_E

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x9840d70


In [3]:
# CLD
# x_angle = 0.030 # rad
# ILD
x_angle = 0.014 # rad
n_threads = 12
prod = False
# prod = True
no_rvec = True
write_outputs = False
# write_outputs = True
# plot_dir_postfix = "-new-cuts"
dataset_path = "data/datasets/pre-selected/checked-test.json"
# output_path = "root://eospublic.cern.ch//eos/experiment/clicdp/data/user/l/lreichen/snapshots3/test"
# output_meta_path = "data/datasets/pre-selected"
# output_meta = f"{output_meta_path}/test.json"
# checked_output_meta = f"{output_meta_path}/checked-test.json"
# output_collections = r"^(?!R2jet_lvecs)\w+$" # ["PandoraPFOs"]
# output_collections = ["PandoraPFOs", "MCParticlesSkimmed", "Refined2Jets", "PFO_ovl_idx", "PFO_not_ovl_idx"]
# plot_dir = f"plots/pre-selection/test{plot_dir_postfix}"
if prod:
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs-min-aa-min-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/min-higgs.json"
    dataset_path = "data/datasets/pre-selected/checked-signal-only.json"
    # output_path = "root://eospublic.cern.ch//eos/experiment/clicdp/data/user/l/lreichen/snapshots3/min-higgs-d"
    # output_meta_path = "data/datasets/pre-selected"
    # output_meta = f"{output_meta_path}/min-higgs-d.json"
    # checked_output_meta = f"{output_meta_path}/checked-min-higgs-d.json"
    # output_collections = ["PandoraPFOs", "MCParticlesSkimmed", "Refined2Jets", "PFO_ovl_idx", "PFO_not_ovl_idx"]
    # plot_dir = "plots/pre-selection/full"
    # plot_dir = f"plots/pre-selection/min-higgs{plot_dir_postfix}"


In [4]:
ROOT.EnableImplicitMT(n_threads)

In [5]:
dataset = Dataset.from_json(dataset_path)

In [6]:
analysis = ObjectSelectionHelper(dataset)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xd87c8d0


In [7]:
analysis.init_categories()
# check if we missed any processes
print(analysis.is_complete_categorisation())
signal_category = ["4f_sw_sl_signal"]

True


In [8]:
ROOT.gStyle.SetOptStat(1)

In [9]:
# import json
# print(json.dumps(analysis._categories, indent=2))

In [10]:
analysis.define_truth_objects(signal_category)

In [11]:
# make nominal iso lep
# try to make iso lep + brems
analysis.Define("iso_lep_idx", "IsolatedElectrons_objIdx.index[0]")
analysis.Define("iso_lep_charge", "PandoraPFOs.charge[iso_lep_idx]")
analysis.Define("iso_lep_lvec", "ROOT::Math::PxPyPzEVector(PandoraPFOs.momentum.x[iso_lep_idx], PandoraPFOs.momentum.y[iso_lep_idx], PandoraPFOs.momentum.z[iso_lep_idx], PandoraPFOs.energy[iso_lep_idx])")
analysis.Define("iso_lep_E", "iso_lep_lvec.energy()")

analysis.Define("n_iso_gammas", "IsolatedPhotons_objIdx.index.size()")
analysis.Define("iso_gamma_idx", "IsolatedPhotons_objIdx.index[0]")
analysis.Define("iso_gamma_lvec", "ROOT::Math::PxPyPzEVector(PandoraPFOs.momentum.x[iso_gamma_idx], PandoraPFOs.momentum.y[iso_gamma_idx], PandoraPFOs.momentum.z[iso_gamma_idx], PandoraPFOs.energy[iso_gamma_idx])")
analysis.Define("iso_lep_gamma_lvec", "n_iso_gammas > 0 ? iso_lep_lvec + iso_gamma_lvec : iso_lep_lvec")

In [12]:
analysis.define_only_on(signal_category, "true_lep_E", "true_lep_lvec.energy()")
analysis.define_only_on(signal_category, "iso_lep_delta_E", "iso_lep_lvec.energy() - true_lep_lvec.energy()")
analysis.define_only_on(signal_category, "iso_lep_gamma_delta_E", "iso_lep_gamma_lvec.energy() - true_lep_lvec.energy()")

analysis.define_only_on(signal_category, "iso_lep_delta_theta", "iso_lep_lvec.Theta() - true_lep_lvec.Theta()")
analysis.define_only_on(signal_category, "iso_lep_gamma_delta_theta", "iso_lep_gamma_lvec.Theta() - true_lep_lvec.Theta()")

analysis.define_only_on(signal_category, "iso_lep_delta_phi", "ROOT::Math::VectorUtil::DeltaPhi(iso_lep_lvec, true_lep_lvec)")
analysis.define_only_on(signal_category, "iso_lep_gamma_delta_phi", "ROOT::Math::VectorUtil::DeltaPhi(iso_lep_gamma_lvec, true_lep_lvec)")

analysis.define_only_on(signal_category, "iso_lep_delta_phi_q", "iso_lep_delta_phi * true_iso_lep_charge")
analysis.define_only_on(signal_category, "iso_lep_gamma_delta_phi_q", "iso_lep_gamma_delta_phi * true_iso_lep_charge")


In [13]:
analysis.book_histogram_1D("iso_lep_delta_E", "iso_lep_delta_E", ("", ";iso lep #Delta E [GeV]", 150, -10., 5.), categories=signal_category)
analysis.book_histogram_1D("iso_lep_gamma_delta_E", "iso_lep_gamma_delta_E", ("", ";iso lep #Delta E [GeV]", 150, -10., 5.), categories=signal_category)

analysis.book_histogram_1D("true_lep_E", "true_lep_E", ("", "true lep E", 125, 0., 125.), categories=signal_category)

analysis.book_histogram_1D("iso_lep_delta_theta", "iso_lep_delta_theta", ("", ";iso lep #Delta #theta [rad]", 150, -0.0005, 0.0005), categories=signal_category)
analysis.book_histogram_1D("iso_lep_gamma_delta_theta", "iso_lep_gamma_delta_theta", ("", ";iso lep #Delta #theta [rad]", 150, -0.0005, 0.0005), categories=signal_category)

analysis.book_histogram_1D("iso_lep_delta_phi", "iso_lep_delta_phi", ("", ";iso lep #Delta #phi [rad]", 150, -0.0005, 0.0005), categories=signal_category)
analysis.book_histogram_1D("iso_lep_gamma_delta_phi", "iso_lep_gamma_delta_phi", ("", ";iso lep #Delta #phi [rad]", 150, -0.0005, 0.0005), categories=signal_category)

analysis.book_histogram_1D("iso_lep_delta_phi_q", "iso_lep_delta_phi_q", ("", ";iso lep #Delta #phi #dot q [rad]", 150, -0.0005, 0.0005), categories=signal_category)
analysis.book_histogram_1D("iso_lep_gamma_delta_phi_q", "iso_lep_gamma_delta_phi_q", ("", ";iso lep #Delta #phi #dot q [rad]", 150, -0.0005, 0.0005), categories=signal_category)

In [14]:
analysis.book_histogram_1D("iso_lep_E", "iso_lep_E", ("", "iso lep E", 125, 0., 125.))

In [15]:
if write_outputs:
    analysis.book_snapshots("events", output_path, output_meta, output_collections, no_rvec=no_rvec)

In [16]:
%%time
analysis.run()

CPU times: user 4.56 s, sys: 394 ms, total: 4.95 s
Wall time: 4.75 s


In [17]:
if write_outputs:
    analysis.check_snapshots("events", output_path, checked_output_meta)

In [18]:
analysis.draw_histogram("true_lep_E", categories=signal_category, draw_legend=False)
analysis.draw_histogram("iso_lep_delta_E", categories=signal_category, draw_legend=False)
analysis.draw_histogram("iso_lep_gamma_delta_E", categories=signal_category, draw_legend=False)

analysis.draw_histogram("iso_lep_delta_theta", categories=signal_category, draw_legend=False)
analysis.draw_histogram("iso_lep_gamma_delta_theta", categories=signal_category, draw_legend=False)

analysis.draw_histogram("iso_lep_delta_phi", categories=signal_category, draw_legend=False)
analysis.draw_histogram("iso_lep_gamma_delta_phi", categories=signal_category, draw_legend=False)

analysis.draw_histogram("iso_lep_delta_phi_q", categories=signal_category, draw_legend=False)
analysis.draw_histogram("iso_lep_gamma_delta_phi_q", categories=signal_category, draw_legend=False)


(<cppyy.gbl.THStack object at 0x13bf8c60>,
 <cppyy.gbl.TCanvas object at 0x1441c240>)

In [19]:
analysis.draw_histogram("iso_lep_E")

(<cppyy.gbl.THStack object at 0x14bbd890>,
 <cppyy.gbl.TCanvas object at 0x14c9a1a0>)